# Targeting Direct Marketing with Amazon SageMaker XGBoost
_**Supervised Learning with Gradient Boosted Trees: A Binary Prediction Problem With Unbalanced Classes**_

---

---

## Contents

1. [Background](#Background)
1. [Prepration](#Preparation)
1. [Data](#Data)
    1. [Exploration](#Exploration)
    1. [Transformation](#Transformation)
1. [Training](#Training)
1. [Hosting](#Hosting)
1. [Evaluation](#Evaluation)
1. [Exentsions](#Extensions)

---

## Background
우편, 이메일, 전화 등을 통한 직접 마케팅은 고객 확보를위한 일반적인 전략입니다. 리소스와 고객의 관심이 제한되어 있기 때문에 목표는 특정 제안에 참여할 가능성이있는 잠재 고객의 하위 집합 만 타겟팅하는 것입니다. 인구 통계, 과거 상호 작용 및 환경 요인과 같이 쉽게 사용할 수있는 정보를 기반으로 잠재 고객을 예측하는 것은 일반적인 기계 학습 문제입니다.

이 노트북은 한 번 이상의 전화 통화 후 고객이 은행에서 정기 예금에 등록할지 예측하는 예제 문제를 제공합니다. 단계는 다음과 같습니다.

* Amazon SageMaker 노트북 준비
* 인터넷에서 Amazon SageMaker로 데이터 다운로드
* Amazon SageMaker 알고리즘에 제공 될 수 있도록 데이터 조사 및 변환
* Gradient Boosting 알고리즘을 사용한 모델 추정
* 모델의 효과 평가
* 지속적인 예측을위한 모델 설정

---

## Preparation

_This notebook was created and tested on an ml.m4.xlarge notebook instance._

Let's start by specifying:

-학습 및 모델 데이터에 사용할 S3 bucket 및 prefix. 노트북 인스턴스, 교육 및 호스팅과 동일한 region 내에 있어야 합니다.
-데이터에 대한 교육 및 호스팅 액세스 권한을 부여하는 데 사용되는 IAM role arn. 이를 만드는 방법은 설명서를 참조하십시오. 노트북 인스턴스, 교육 및 / 또는 호스팅에 둘 이상의 역할이 필요한 경우 boto regexp를 적절한 전체 IAM role arn 문자열로 바꾸십시오.

In [ ]:
conda update pandas

In [ ]:
import sagemaker
bucket=sagemaker.Session().default_bucket()
prefix = 'sagemaker/DEMO-xgboost-dm'
 
# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

Now let's bring in the Python libraries that we'll use throughout the analysis

In [ ]:
import numpy as np                                # For matrix operations and numerical processing
import pandas as pd                               # For munging tabular data
import matplotlib.pyplot as plt                   # For charts and visualizations
from IPython.display import Image                 # For displaying images in the notebook
from IPython.display import display               # For displaying outputs in the notebook
from time import gmtime, strftime                 # For labeling SageMaker models, endpoints, etc.
import sys                                        # For writing outputs to notebook
import math                                       # For ceiling function
import json                                       # For parsing hosting outputs
import os                                         # For manipulating filepath names
import sagemaker 
import zipfile     # Amazon SageMaker's Python SDK provides many helper functions

In [ ]:
pd.__version__

Make sure pandas version is set to 1.2.4 or later. If it is not the case, restart the kernel before going further

---

## Data
Let's start by downloading the [direct marketing dataset](https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip) from the sample data s3 bucket. 

\[Moro et al., 2014\] S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems, Elsevier, 62:22-31, June 2014


In [ ]:
!wget https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip

with zipfile.ZipFile('bank-additional.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

Now lets read this into a Pandas data frame and take a look.

In [ ]:
data = pd.read_csv('./bank-additional/bank-additional-full.csv')
pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 20)         # Keep the output on one page
data

데이터에 대해 얘기하면 높은 수준에서 우리는 다음을 볼 수 있습니다.

* 40K 이상의 고객 기록과 각 고객에 대해 20 개의 features가 있습니다.
* features는 numeric과 categorical 형태로 혼합되어 있습니다.
* 데이터가 최소한 `time`와 `contact`별로 정렬 된 것으로 보입니다.

_**각 features에 대한 사양:**_

*Demographics:*
* `age`: 고객의 나이 (numeric)
* `job`: 작업 유형 (categorical : 'admin.', 'services', ...)
* `marital`: 결혼 여부 (categorical: 'married', 'single', ...)
* `education` : 교육 수준 (categorical: 'basic.4y', 'high.school', ...)

*Past customer events:*
* `default`:  신용이 채무불이행입니까? (categorical: 'no', 'unknown', ...)
* `housing`: 주택 대출이 있습니까? (categorical: 'no', 'yes', ...)
* `loan`: 개인 대출이 있습니까? (categorical: 'no', 'yes', ...)

*Past direct marketing contacts:*
* `contact`: 연락처 통신 유형 (categorical: 'cellular', 'telephone', ...)
* `month` : 연중 마지막 연락 월 (categorical: 'may', 'nov', ...)
* `day_of_week` : 마지막 연락 요일 (categorical: 'mon', 'fri', ...)
* `duration` : 마지막 접촉 시간 (초) (numeric). 중요 참고 : If duration = 0 then `y` = 'no'.
 
*Campaign information:*
* `campaign`: 이 캠페인 및 이 고객에 대해 수행된 연락 횟수 (numeric, includes last contact)
* `pdays` : 이전 캠페인에서 고객이 마지막으로 연락 한 후 경과 한 일수 (numeric)
* `previous`:이 캠페인 이전 및 이 고객에 대해 수행 된 연락 횟수 (numeric)
* `poutcome` : 이전 마케팅 캠페인 결과 (categorical: 'nonexistent','success', ...)

*External environment factors:*
* `emp.var.rate` : 고용 변동률-분기 별 지표 (numeric)
* `cons.price.idx` : 소비자 물가 지수-월간 지표 (numeric)
* `cons.conf.idx` : 소비자 신뢰 지수-월간 지표 (numeric)
* `euribor3m` : Euribor 3 개월 요금-일별 표시기 (numeric)
* `nr.employed`: 직원 수-분기 별 지표 (numeric)

*Target variable:*
* `y` : 고객이 정기 예금을 신청 했습니까? (binary: 'yes','no')

### Exploration
Let's start exploring the data.  First, let's understand how the features are distributed.

In [ ]:
# Frequency tables for each categorical feature
for column in data.select_dtypes(include=['object']).columns:
    display(pd.crosstab(index=data[column], columns='% observations', normalize='columns'))

# Histograms for each numeric features
display(data.describe())
%matplotlib inline
hist = data.hist(bins=30, sharey=True, figsize=(10, 10))

Notice that:

* target variable `y`값의 거의 90 %가 "no"이므로 대부분의 고객은 정기 예금을 신청하지 않았습니다.
* 많은 예측을 위한 Feature들은 "unknown" 값을 가집니다. 경우에 따라 다른 값보다 더 많이 있는 경우도 있습니다. 우리는 "unknown"의 값을 발생시키는 원인이 무엇인지와 이를 어떻게 처리해야 하는지 신중하게 생각해야 합니다.
  * "unknown"이 고유한 카테고리로 포함 되더라도, 해당 feature의 다른 카테고리 중 하나에 속할 가능성이 있다는 점을 감안한다면 그것은 무엇을 의미할까요?
* 많은 feature에는 관측치가 거의 없는 카테고리가 있습니다. target 결과에 매우 예측 가능한 작은 범주를 찾으면 이에 대해 일반화 할 충분한 증거가 있습니까?
* Contact timing은 특히 skew됩니다. 5 월에는 거의 1/3, 12 월에는 1 % 미만입니다. 이것이 다음 12 월 target variable을 예측하는 데 어떤 의미가 있습니까?
* numeric features에는 결측값이 없습니다. 또는 결측값이 이미 대치되었습니다.
  * `pdays`는 거의 모든 고객에 대해 1000에 가까운 값을 취합니다. 이전 연락이 없음을 나타내는 placeholder 값일 수 있습니다.
* 일부 numeric features은 매우 long tail한 feature을 가지고 있습니다. 몇 가지 관찰값들은 극단적 큰 값으로 다르게 처리가 필요할까요?
* 여러 numeric features (특히 거시 경제적 특성)이 서로 다른 버킷에서 발생합니다. 이들은 categorical형으로 취급되어야 합니까?

다음으로, 우리의 기능이 우리가 예측하려는 대상과 어떻게 관련되는지 살펴 보겠습니다.

* Almost 90% of the values for our target variable `y` are "no", so most customers did not subscribe to a term deposit.
* Many of the predictive features take on values of "unknown".  Some are more common than others.  We should think carefully as to what causes a value of "unknown" (are these customers non-representative in some way?) and how we that should be handled.
  * Even if "unknown" is included as it's own distinct category, what does it mean given that, in reality, those observations likely fall within one of the other categories of that feature?
* Many of the predictive features have categories with very few observations in them.  If we find a small category to be highly predictive of our target outcome, do we have enough evidence to make a generalization about that?
* Contact timing is particularly skewed.  Almost a third in May and less than 1% in December.  What does this mean for predicting our target variable next December?
* There are no missing values in our numeric features.  Or missing values have already been imputed.
  * `pdays` takes a value near 1000 for almost all customers.  Likely a placeholder value signifying no previous contact.
* Several numeric features have a very long tail.  Do we need to handle these few observations with extremely large values differently?
* Several numeric features (particularly the macroeconomic ones) occur in distinct buckets.  Should these be treated as categorical?

Next, let's look at how our features relate to the target that we are attempting to predict.

In [ ]:
for column in data.select_dtypes(include=['object']).columns:
    if column != 'y':
        display(pd.crosstab(index=data[column], columns=data['y'], normalize='columns'))

for column in data.select_dtypes(exclude=['object']).columns:
    print(column)
    hist = data[[column, 'y']].hist(by='y', bins=30)
    plt.show()

Notice that:

* "blue-collar", "married" 이고 채무불이행 (default)가 "unknown" 상태, "telephone"로 연락했으며, 연락월이 "may" 에 해당하는 고객은 정기예금 신청에 대해 "no"보다 "yes"의 비율이 상당히 낮습니다.
* numeric variables에 대한 분포는 "yes" 및 "no" 의 신청 그룹에 따라 상이하지만 상관관계가 간단하거나 명확하지 않을 수 있습니다.


Now let's look at how our features relate to one another.

In [ ]:
display(data.corr())
pd.plotting.scatter_matrix(data, figsize=(12, 12))
plt.show()

Notice that:

* Features의 상호 관계는 서로서로 크게 다릅니다. 일부는 매우 음의 상관 관계를 가지며 다른 일부는 매우 양의 상관 관계를 가집니다.
* Features 간의 관계는 대부분의 경우 non-linear이며 discrete입니다.

### Transformation

Data cleansing은 거의 모든 기계 학습 프로젝트의 일부입니다. 잘못 수행하면 가장 큰 위험을 초래하고 프로세스에서 더 주관적인 측면 중 하나입니다. 몇 가지 일반적인 기술은 다음과 같습니다.

* 결측값 처리 : 일부 기계 학습 알고리즘은 결측값을 처리 할 수 있지만 대부분은 처리하지 않습니다. 옵션은 다음과 같습니다.
 * 결측값이 있는 관측치 제거 : 이것은 관측치의 아주 작은 부분에만 불완전한 정보가 있는 경우 잘 작동합니다.
 * 결측값을 가진 features 제거 : 많은 수의 결측값을 가진 적은 수의 features에 대해 이 방법이 잘 작동합니다.
 * 결측값 대치 : 전체 [books](https://www.amazon.com/Flexible-Imputation-Missing-Interdisciplinary-Statistics/dp/1439868247)이 이와 관련된 주제에 대해 작성되었지만 일반적인 선택은 결측값을 다음으로 대체합니다. 해당 열의 비결측값의 최빈값 또는 평균.
* categorical형을 numeric로 변환 : 가장 일반적인 방법은 one-hot 인코딩으로 변환합니다.
* 이상하게 분포된 데이터 : Gradient Boosted Trees와 같은 비선형 모델의 경우 이는 매우 제한적이지만 회귀와 같은 모수 모델은 고도로 치우친 데이터를 제공 할 때 매우 부정확한 추정치를 생성 할 수 있습니다. 어떤 경우에는 단순히 특성의 자연 로그를 취하는 것만으로도 더 정규분포된 데이터를 생성하는 데 충분합니다. 다른 경우에는 값을 discrete 범위로 버킷화하는 것이 도움이 됩니다. 그런 다음 이러한 버킷을 categorical형 변수로 취급하고 one-hot encoding된 경우 모델에 포함할 수 있습니다.
* 더 복잡한 데이터 유형 처리 : 이미지, 텍스트 또는 데이터를 다양한 입자로 관리하는 것은 다른 노트북 템플릿에 남겨집니다.

운 좋게도 이러한 측면 중 일부는 이미 우리를 위해 처리되었으며, 우리가 소개하는 알고리즘은 희소하거나 이상하게 분산 된 데이터를 잘 처리하는 경향이 있습니다. 따라서 전처리를 간단하게 유지합니다.

In [ ]:
data['no_previous_contact'] = np.where(data['pdays'] == 999, 1, 0)                                 # Indicator variable to capture when pdays takes a value of 999
data['not_working'] = np.where(np.in1d(data['job'], ['student', 'retired', 'unemployed']), 1, 0)   # Indicator for individuals not actively employed
model_data = pd.get_dummies(data)                                                                  # Convert categorical variables to sets of indicators

모델을 구축하기 전에 스스로에게 물어볼 또 다른 질문은 특정 features가 최종 사용 사례에 가치를 더할 것인지 여부입니다. 예를 들어, 최상의 예측을 제공하는 것이 목표라면 예측 순간에 해당 데이터에 액세스 할 수 있어야 합니다. 비가 오는 것을 아는 것은 우산 판매에 대해 매우 예측 가능하지만 우산 재고를 계획 할만큼 멀리 날씨를 예측하는 것은 날씨에 대한 지식없이 우산 판매를 예측하는 것만큼 어려울 수 있습니다. 따라서 이것을 모델에 포함하면 잘못된 정확성을 얻을 수 있습니다.

이 논리에 따라 향후 예측에서 입력으로 사용하기 위해 고정밀로 예측해야 하는 경제적 특성과 `duration`을 데이터에서 제거해야 합니다.

이전 분기의 경제 지표 값을 사용한다고 하더라도 이 값은 나중에 contact한 것만큼 다음 분기 초에 contact한 잠재 고객과 관련이 없을 가능성이 높습니다.

In [ ]:
model_data = model_data.drop(['duration', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed'], axis=1)

새로운 데이터에 대한 target value를 예측하는 것이 주요 목표인 모델을 구축 할 때 과적합을 이해하는 것이 중요합니다. 지도 학습 모델은 주어진 데이터에서 목표 값과 실제 값에 대한 예측 간의 오류를 최소화하도록 설계되었습니다. 이 마지막 부분은 더 큰 정확성을 추구하는 경우가 많기 때문에 머신 러닝 모델은 표시된 데이터 내에서 사소한 특이성을 포착하는 쪽으로 편향됩니다. 이러한 특이성은 후속 데이터에서 반복되지 않습니다. 즉, 훈련 단계에서 더 정확한 예측을 할 수 있으나 이러한 예측이 실제로 덜 정확해질 수 있습니다.

이를 방지하는 가장 일반적인 방법은 모델이 학습된 데이터에 대한 적합성뿐만 아니라 "새로운" 데이터에 대해서도 판단되어야 한다는 개념으로 모델을 구축하는 것입니다. 이를 운영하는 데는 여러 가지 방법, holdout validation, cross-validation, leave-one-out validation 등이 있습니다. 목적을 위해 데이터를 무작위로 3 개의 고르지 않은 그룹으로 분할합니다. 이 모델은 데이터의 70 %에 대해 학습 된 후 데이터의 20 %에 대해 평가되어 "새로운"데이터에 대해 우리가 원하는 정확도를 추정하고, 10 %는 최종 테스트로 사용합니다.



In [ ]:
train_data, validation_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data)), int(0.9 * len(model_data))])   # Randomly sort the data then split out first 70%, second 20%, and last 10%

Amazon SageMaker의 XGBoost 컨테이너는 libSVM 또는 CSV 데이터 형식의 데이터를 기대합니다. 이 예에서는 CSV를 사용합니다. 첫 번째 열은 대상 변수 여야하며 CSV에는 헤더가 포함되지 않아야합니다. 또한 반복적이지만 train | validation | test 분할 이전이 아닌 이후에이 작업을 수행하는 것이 가장 쉽습니다. 이렇게하면 무작위 재정렬로 인한 정렬 불량 문제를 방지 할 수 있습니다.

In [ ]:
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)
pd.concat([validation_data['y_yes'], validation_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('validation.csv', index=False, header=False)

In [ ]:
train_data_final = pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1)
train_data_final.to_csv('train_header.csv', index=False, header=True)

test_data_final = pd.concat([validation_data['y_yes'], validation_data.drop(['y_no', 'y_yes'], axis=1)], axis=1)
test_data_final.to_csv('validation_header.csv', index=False, header=True)

Now we'll copy the file to S3 for Amazon SageMaker's managed training to pickup.

In [ ]:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')

In [ ]:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train_header/train_header.csv')).upload_file('train_header.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation_header/validation_header.csv')).upload_file('validation_header.csv')

---

## End of Lab 1


---

## Training
이제 우리는 대부분의 features이 skewed 분포를 가지고 있고 일부는 서로 높은 상관 관계가 있으며 일부는 target variable와 비선형 관계를 갖는 것처럼 보입니다. 또한 미래의 잠재 고객을 대상으로하는 경우 해당 잠재 고객이 대상이 된 이유를 설명 할 수있는 것보다 우수한 예측 정확도가 선호됩니다. 이러한 측면을 종합하면 gradient boosted trees가 좋은 후보 알고리즘으로 만듭니다.

알고리즘을 이해하는 데는 몇 가지 복잡한 점이 있지만 높은 수준에서 gradient boosted trees는 많은 단순 모델의 예측을 결합하여 작동하며, 각 모델은 이전 모델의 약점을 해결하려고합니다. 이렇게하면 단순한 모델 모음이 실제로 크고 복잡한 모델보다 성능이 뛰어납니다. 다른 Amazon SageMaker 노트북은 gradient boosted trees에 대해 자세히 설명하고 유사한 알고리즘과 어떻게 다른지 설명합니다.

`xgboost`는 gradient boosted trees를 위한 매우 인기있는 오픈 소스 패키지입니다. 계산적으로 강력하고 모든 기능을 갖추고 있으며 많은 기계 학습 대회에서 성공적으로 사용되었습니다. Amazon SageMaker의 관리 형 분산 교육 프레임 워크를 사용하여 교육 된 간단한 `xgboost`모델부터 시작하겠습니다.

먼저 Amazon SageMaker의 XGBoost 구현을위한 ECR 컨테이너 위치를 지정해야 합니다.

In [ ]:
container = sagemaker.image_uris.retrieve(region=boto3.Session().region_name, framework='xgboost', version='latest')

그런 다음 CSV 파일 형식으로 학습하기 때문에 학습 함수가 S3의 파일에 대한 포인터로 사용할 수있는`s3_input`을 생성하고 콘텐츠 유형이 CSV임을 지정합니다.

In [ ]:
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train/train.csv'.format(bucket, prefix), content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/validation/validation.csv'.format(bucket, prefix), content_type='csv')

먼저 estimator에 training parameters를 지정해야합니다. 여기에는 다음이 포함됩니다.
1. `xgboost` 알고리즘 컨테이너
1. 사용할 IAM 역할
1. 훈련 인스턴스 유형 및 개수
1. 출력 데이터의 S3 위치
1. 알고리즘의 hyperparameters

그리고 다음을 지정하는 `.fit ()` 함수 :
1. output data를 위한 S3 location. 이 경우 전달되는 training 및 validation 세트에 대해서도 모두 있습니다.

In [ ]:
sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    instance_count=1, 
                                    instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100)

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation}) 

---

## Hosting
데이터에 대해 `xgboost`알고리즘을 학습 했으므로 이제 실시간 엔드 포인트 뒤에서 호스팅되는 모델을 배포 해 보겠습니다.

In [ ]:
xgb_predictor = xgb.deploy(initial_instance_count=1,
                           instance_type='ml.m4.xlarge')

---

## Evaluation
머신 러닝 모델의 성능을 비교하는 방법은 여러 가지가 있지만 실제 값과 예측 값을 비교하는 것부터 시작하겠습니다. 이 경우, 우리는 단순히 고객이 정기 예금에 가입했는지 (`1`) 아니면 가입하지 않았는지 (`0`) 예측하는 것이므로 간단한 confusion matrix이 생성합니다.

먼저 엔드 포인트로 데이터를 전달하고 수신하는 방법을 결정해야합니다. 데이터는 현재 노트북 인스턴스의 메모리에 NumPy 배열로 저장됩니다. HTTP POST 요청으로 전송하기 위해 CSV 문자열로 직렬화 한 다음 결과 CSV를 디코딩합니다.

*Note : CSV 형식을 사용한 inference의 경우 SageMaker XGBoost에서 데이터에 target variable가 포함되지 않아야 합니다. *

In [ ]:
xgb_predictor.serializer = sagemaker.serializers.CSVSerializer()

이제 간단한 함수를 사용하여 다음을 수행합니다.
1. test dataset을 반복합니다.
1. rows의 미니 배치로 분할
1. 해당 미니 배치를 CSV 문자열 페이로드로 변환합니다 (먼저 dataset에서 target variable를 삭제합니다).
1. XGBoost endpoint를 호출하여 미니 배치 예측을 검색합니다.
1. 예측을 수집하고 모델이 제공하는 CSV output에서 NumPy 배열로 변환

In [ ]:
def predict(data, predictor, rows=500 ):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

predictions = predict(test_data.drop(['y_no', 'y_yes'], axis=1).to_numpy(), xgb_predictor)

Now we'll check our confusion matrix to see how well we predicted versus actuals.

In [ ]:
pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions), rownames=['actuals'], colnames=['predictions'])

In [ ]:
test_data.groupby('y_yes').count()

따라서 약 4000 명의 잠재 고객 중 154 명이 가입한다고 예측하였고, 101 명이 실제로 가입하였습니다. 우리는 또한 우리가 예측하지 못했던 구독자들이 382명이 있었습니다. 이는 바람직하지 않지만 이를 개선하기 위해 모델을 조정할 수 있으며 조정해야합니다. 가장 중요한 것은 최소한의 노력으로 우리 모델이 [here](http://media.salford-systems.com/video/tutorial/2015/targeted_marketing.pdf)에 게시 된 것과 유사한 정확도를 생성했다는 점입니다.

_Note : 알고리즘의 하위 샘플에 임의의 요소가 있기 때문에 결과가 위에 쓰여진 텍스트와 약간 다를 수 있습니다._

## Automatic model Tuning (optional)
hyperparameter tuning이라고도 하는 Amazon SageMaker automatic model tuning은 지정한 hyperparameter 범위와 알고리즘을 사용하여 데이터 세트에서 많은 훈련 작업을 실행하여 최상의 모델 버전을 찾습니다. 그런 다음 선택한 측정 항목으로 측정된 대로 최상의 성능을 발휘하는 모델을 만드는 hyperparameter을 선택합니다.
예를 들어 마케팅 데이터 세트에서 이진 분류 문제를 풀고 싶다고 가정합니다. 목표는 XGBoost 알고리즘 모델을 학습하여 알고리즘의 곡선 아래 영역 (auc) 메트릭을 최대화하는 것입니다. 최고의 모델을 훈련시키는 데 사용할 eta, alpha, min_child_weight 및 max_depth 등의 hyperparameter의 값을 모릅니다. 이러한 hyperparameter에 대한 최적의 값을 찾으려면 Amazon SageMaker hyperparameter의 튜닝이 검색하는 값 범위를 지정하여 선택한 객관적인 지표로 측정 한대로 최상의 성능을 발휘하는 훈련 작업을 생성하는 값의 조합을 찾을 수 있습니다. hyperparameter tuning은 지정한 범위의 hyperparameter 값을 사용하는 학습 작업을 시작하고 auc가 가장 높은 학습 작업을 반환합니다.

In [ ]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
hyperparameter_ranges = {'eta': ContinuousParameter(0, 1),
                         'min_child_weight': ContinuousParameter(1, 10),
                         'alpha': ContinuousParameter(0, 2),
                         'max_depth': IntegerParameter(1, 10)}


In [ ]:
objective_metric_name = 'validation:auc'

In [ ]:
tuner = HyperparameterTuner(xgb,
                            objective_metric_name,
                            hyperparameter_ranges,
                            max_jobs=4,
                            max_parallel_jobs=2)


In [ ]:
tuner.fit({'train': s3_input_train, 'validation': s3_input_validation})

In [ ]:
boto3.client('sagemaker').describe_hyper_parameter_tuning_job(
HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']

In [ ]:
# return the best training job name
tuner.best_training_job()

In [ ]:
#  Deploy the best trained or user specified model to an Amazon SageMaker endpoint
tuner_predictor = tuner.deploy(initial_instance_count=1,
                           instance_type='ml.m4.xlarge')

In [ ]:
# Create a serializer
tuner_predictor.serializer = sagemaker.serializers.CSVSerializer()

In [ ]:
# Predict
predictions = predict(test_data.drop(['y_no', 'y_yes'], axis=1).to_numpy(),tuner_predictor)

In [ ]:
# Collect predictions and convert from the CSV output our model provides into a NumPy array
pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions), rownames=['actuals'], colnames=['predictions'])

## Amazon SageMaker Clarify
잠재적인 편향을 감지하고 이러한 모델이 예측을 수행하는 방법을 설명하여 기계학습 모델을 개선하는데 도움이 됩니다. SageMaker Clarify에서 제공하는 fairness와 explainability 기능은 AWS 고객이 신뢰할 수 있고 이해하기 쉬운 기계학습 모델을 구축 할 수 있도록 한 걸음 더 나아갑니다. 이 제품은 다음 작업에 도움이 되는 도구와 함께 제공됩니다.

* ML lifecycle의 각 단계에서 발생할 수있는 biases를 측정합니다.(추론을 위해 배포된 ML 모델의 데이터 수집, 모델 교육 및 tuning, 모니터링)
* Risk/Compliance 팀과 외부 규제 기관을 대상으로 하는 모델 거버넌스 보고서를 생성합니다.
* 예측을 평가하는 데 사용되는 데이터, 모델 및 모니터링에 대한 설명을 제공합니다.

1. SageMaker Clarify를 이해하는 데 필요한 주요 용어 및 개념
1. 데이터 세트의 학습 전 bias과 모델의 학습 후 bias 측정
1. 모델의 결정에 대한 다양한 입력 features의 중요성 설명
1. 인스턴스가 설정된 경우 SageMaker Studio를 통해 보고서에 액세스합니다.

이 과정에서 노트북은 먼저 교육 데이터 세트를 사용하여 [SageMaker XGBoost](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html) 모델을 학습한 다음 SageMaker Clarify를 사용하여 CSV 형식의 테스트 데이터 세트를 분석합니다. SageMaker Clarify는 [SageMaker JSONLines dense 형식](https://docs.aws.amazon.com/sagemaker/latest/dg/cdf-inference.html#common-in-formats)의 데이터 세트 분석도 지원합니다. [another notebook](https://github.com/aws/amazon-sagemaker-examples/blob/master/sagemaker_processing/fairness_and_explainability/fairness_and_explainability_jsonlines_format.ipynb).

### Data inspection

다양한 기능의 분포에 대한 히스토그램을 플로팅하는 것은 데이터를 시각화하는 좋은 방법입니다. _sensitive_로 간주 될 수 있는 몇 가지 기능을 플로팅해 보겠습니다.
첫 번째 플롯에서는 전체적으로 여성 응답자가 더 적지 만 특히 긍정적 인 결과에서 응답자의 ~$\frac{1}{7}$th를 구성합니다.


In [ ]:
training_data=pd.read_csv('./bank-additional/bank-additional-full.csv')

In [ ]:
training_data

In [ ]:
training_data['job'].value_counts().sort_values().plot(kind='bar', title='Counts of Job', rot=45)

In [ ]:
training_data['job'].where(training_data['y']=='yes').value_counts().sort_values().plot(kind='bar', title='Counts of Job if y == yes', rot=45)

In [ ]:
training_data['job'].where(training_data['y']=='no').value_counts().sort_values().plot(kind='bar', title='Counts of Job if y == no', rot=45)

In [ ]:
model_name = 'DEMO-clarify-model'
model = xgb.create_model(name=model_name)
container_def = model.prepare_container_def()
sess.create_model(model_name,
                     role,
                     container_def)

In [ ]:
from sagemaker import clarify
clarify_processor = clarify.SageMakerClarifyProcessor(role=role,
                                                      instance_count=1,
                                                      instance_type='ml.m5.xlarge',
                                                      sagemaker_session=sess)

In [ ]:
train_uri='s3://{}/{}/train_header/train_header.csv'.format(bucket, prefix)

### Detecting Bias
SageMaker Clarify는 다양한 지표를 사용하여 가능한 사전 및 사후 훈련 편향을 감지하는 데 도움이 됩니다.
#### Writing DataConfig and ModelConfig
`DataConfig` 객체는 데이터 I/O에 대한 몇 가지 기본 정보를 SageMaker Clarify에 전달합니다. 입력 데이터 세트를 찾을 위치, 출력을 저장할 위치, 대상 열 (`label`), 헤더 이름 및 데이터 세트 유형을 지정합니다.

In [ ]:
train_uri

In [ ]:
pd.read_csv('train_header.csv')

In [ ]:
bias_report_output_path = 's3://{}/{}/clarify-bias'.format(bucket, prefix)
bias_data_config = clarify.DataConfig(s3_data_input_path=train_uri,
                                      s3_output_path=bias_report_output_path,
                                      label='y_yes',
                                      headers=train_data_final.columns.to_list(),
                                      dataset_type='text/csv')

`ModelConfig` 객체는 학습된 모델에 대한 정보를 전달합니다. 프로덕션 모델에 대한 추가 트래픽을 방지하기 위해 SageMaker Clarify는 처리시 전용 엔드 포인트를 설정하고 해제합니다.
* `instance_type`과 `instance_count`는 SageMaker Clarify가 처리하는 동안 모델을 실행하는 데 사용되는 선호하는 인스턴스 유형 및 인스턴스 수를 지정합니다. 테스트 데이터 세트는 작기 때문에 단일 표준 인스턴스로도 이 예제를 실행할 수 있습니다. 복잡한 대규모 데이터 세트가 있는 경우 더 나은 인스턴스 유형을 사용하여 속도를 높이거나 인스턴스를 더 추가하여 Spark 병렬화를 활성화 할 수 있습니다.
* `accept_type`은 endpoint response payload 형식을,`content_type`은 endpoint에 대한 request payload 형식을 나타냅니다.

In [ ]:
model_config = clarify.ModelConfig(model_name=model_name,
                                   instance_type='ml.m5.xlarge',
                                   instance_count=1,
                                   accept_type='text/csv',
                                   content_type='text/csv')

`ModelPredictedLabelConfig`는 예측 형식에 대한 정보를 제공합니다. XGBoost 모델은 샘플 확률을 출력하므로, SageMaker Clarify는 엔드 포인트를 호출한 다음 `probability_threshold`를 사용하여 편향 분석을 위해 확률을 이진 레이블로 변환합니다. 임계 값을 초과하는 예측은 라벨 값 `1`로 해석되고 라벨 값 `0`보다 작거나 같습니다.

In [ ]:
predictions_config = clarify.ModelPredictedLabelConfig(probability_threshold=0.5)

#### Writing BiasConfig
SageMaker Clarify는 민감한 열(`facets`)이 무엇인지, 민감한 features (`facet_values_or_threshold`)이 무엇인지, 원하는 결과 (`label_values_or_threshold`)에 대한 정보도 필요합니다.
SageMaker Clarify는`facet_values_or_threshold` 및`label_values_or_threshold`에 대한 categorical 및 continuous 데이터를 모두 처리 할 수 있습니다. 이 경우 categorical 데이터를 사용하고 있습니다.

이 정보는 `BiasConfig` API에서 지정합니다. `group_name`은 Simpson의 역설과 관련하여 CDDL (Conditional Demographic Disparity in Labels) 및 CDDPL (Conditional Demographic Disparity in Predicted Labels) 측정을 위한 하위 그룹을 형성하는 데 사용됩니다.

In [ ]:
bias_config = clarify.BiasConfig(label_values_or_threshold=[1],
                                 facet_name='job_blue-collar',
                                 facet_values_or_threshold=[1],
                                 group_name='age')

#### Pre-training Bias
모델 학습이 발생하기 전에 데이터에 bias가 존재할 수 있습니다. 학습이 시작되기 전에 데이터의 bias를 검사하면 데이터 수집 격차를 감지하고 feature engineering에서 데이터가 반영 할 수 있는 social biases을 이해하는 데 도움이 될 수 있습니다.

pre-training bias metrics 계산에는 훈련 된 모델이 필요하지 않습니다.

#### Post-training Bias
post-training bias metrics을 계산하려면 훈련된 모델이 필요합니다.

편향되지 않은 학습 데이터 (편향 메트릭으로 측정 된 공정성 개념에 의해 결정됨)는 학습 후에도 여전히 편향된 모델 예측을 초래할 수 있습니다. 이것이 발생하는지 여부는 하이퍼 파라미터 선택을 포함한 여러 요인에 따라 다릅니다.


이러한 옵션은`run_pre_training_bias()`및`run_post_training_bias()`와 함께 개별적으로 실행하거나 아래와 같이`run_bias ()`와 동시에 실행할 수 있습니다.

In [ ]:
clarify_processor.run_bias(data_config=bias_data_config,
                           bias_config=bias_config,
                           model_config=model_config,
                           model_predicted_label_config=predictions_config,
                           pre_training_methods='all',
                           post_training_methods='all')

In [ ]:
test_features = test_data_final.drop(['y_yes'], axis = 1)

In [ ]:
bias_report_output_path

### Explaining Predictions
모델이 결정을 내린 이유에 대한 설명을 요구하는 비즈니스 요구와 법적 규정이 확대되고 있습니다. SageMaker Clarify는 SHAP를 사용하여 각 입력 기능이 최종 결정에 미치는 기여를 설명합니다.

Kernel SHAP 알고리즘에는 baseline (background 데이터 세트라고도 함)이 필요합니다. 기준 데이터 세트 유형은 `DataConfig`의 `dataset_type`과 동일해야 하며 Baseline 샘플에는 features만 포함되어야 합니다. 정의에 따라 `baseline`은 기준 데이터 세트 파일에 대한 S3 URI이거나 내부 샘플 목록이어야 합니다. 이 경우 후자를 선택하고 테스트 데이터 세트의 첫 번째 샘플을 목록에 넣습니다.

In [ ]:
shap_config = clarify.SHAPConfig(baseline=[test_features.iloc[0].values.tolist()],
                                 num_samples=15,
                                 agg_method='mean_abs',
                                 save_local_shap_values=False)

explainability_output_path = 's3://{}/{}/clarify-explainability'.format(bucket, prefix)
explainability_data_config = clarify.DataConfig(s3_data_input_path=train_uri,
                                s3_output_path=explainability_output_path,
                                label='y_yes',
                                headers=train_data_final.columns.to_list(),
                                dataset_type='text/csv')

In [ ]:
clarify_processor.run_explainability(data_config=explainability_data_config,
                                     model_config=model_config,
                                     explainability_config=shap_config)

---

## Extensions
이 예에서는 비교적 작은 데이터 세트를 분석했지만 훨씬 더 큰 문제에 쉽게 적용 할 수있는 분산, 관리형 학습 및 실시간 모델 호스팅과 같은 Amazon SageMaker 기능을 활용했습니다. 예측 정확도를 더욱 향상시키기 위해 예측 임계 값을 조정하여 false-positives과 false-negatives의 조합을 변경하거나 hyperparameter tuning과 같은 기술을 탐색 할 수 있습니다. 실제 시나리오에서는 수동으로 기능을 엔지니어링하는 데 더 많은 시간을 할애하고 초기 데이터 세트에서 사용할 수없는 고객 정보를 포함 할 추가 데이터 세트를 찾을 가능성이 높습니다.

### (Optional) Clean-up

If you are done with this notebook, please run the cell below.  This will remove the hosted endpoint you created and avoid any charges from a stray instance being left on.

In [ ]:
xgb_predictor.delete_endpoint(delete_endpoint_config=True)

In [ ]:
tuner_predictor.delete_endpoint(delete_endpoint_config=True)